In [9]:
# notebooks/10_clean_translation_memory.ipynb

# ===================================================================
# Cell 1: 라이브러리 import 및 환경 설정
# ===================================================================

import os
import sys
from datetime import datetime

# src 폴더를 파이썬 경로에 추가
sys.path.append(os.path.abspath('../src'))

# 프로젝트 모듈들
from config import SUPPORTED_LANGUAGE_PAIRS, MONGO_CONNECTION_STRING, DB_NAME, PROCESSED_DIR
from processors.tm_cleaner import TMCleaner, clean_all_language_pairs, get_cleaning_stats
from pymongo import MongoClient

print("✅ 라이브러리 import 완료")
print(f"🌐 지원 언어 쌍: {SUPPORTED_LANGUAGE_PAIRS}")
print(f"🗄️ Database: {DB_NAME}")

import pandas as pd

from pymongo import MongoClient

# CSV 출력 디렉토리 확인/생성
csv_output_dir = os.path.join(PROCESSED_DIR, "clean_tm_csv")
os.makedirs(csv_output_dir, exist_ok=True)

print("✅ 라이브러리 import 완료")
print(f"📁 CSV 출력 경로: {csv_output_dir}")


✅ 라이브러리 import 완료
🌐 지원 언어 쌍: [('en', 'ko'), ('en', 'ja')]
🗄️ Database: aem_qa_system
✅ 라이브러리 import 완료
📁 CSV 출력 경로: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/3_processed/clean_tm_csv


In [ ]:
# ===================================================================
# Cell 2: Raw TM 현황 확인
# ===================================================================

def preview_raw_tm_samples(lang_suffix: str, limit: int = 5):
    """Raw TM 샘플 미리보기"""
    client = MongoClient(MONGO_CONNECTION_STRING)
    db = client[DB_NAME]
    raw_tm_collection = db[f"translation_memory_{lang_suffix}"]
    
    total_count = raw_tm_collection.count_documents({})
    print(f"🔍 [{lang_suffix}] Raw TM 현황: 총 {total_count:,}개")
    
    if total_count == 0:
        print("   ⚠️ Raw TM 데이터가 없습니다.")
        return
    
    print(f"   📋 샘플 {limit}개:")
    samples = list(raw_tm_collection.find({}).limit(limit))
    
    for i, sample in enumerate(samples, 1):
        source = sample.get('source_text', '')[:80]
        target = sample.get('target_text', '')[:80]
        print(f"   {i}. Source: '{source}{'...' if len(sample.get('source_text', '')) > 80 else ''}'")
        print(f"      Target: '{target}{'...' if len(sample.get('target_text', '')) > 80 else ''}'")
        print(f"      Page: {sample.get('page_path', 'N/A')}")
        print()

# 현재 Raw TM 현황 확인
print("📊 Raw TM 현황 확인:")
for source_lang, target_lang in SUPPORTED_LANGUAGE_PAIRS:
    lang_suffix = f"{source_lang}_{target_lang}"
    print(f"\n--- {source_lang.upper()}-{target_lang.upper()} ---")
    preview_raw_tm_samples(lang_suffix, limit=3)




In [ ]:
# ===================================================================
# Cell 3: 단일 언어 쌍 정제 (EN-KO)
# ===================================================================

print("=" * 60)
print("🇰🇷 한국어 번역 메모리 정제 시작")
print("=" * 60)

# TM Cleaner 초기화
cleaner = TMCleaner()

# EN-KO TM 정제 실행
ko_result = cleaner.clean_translation_memory("en_ko")

print(f"\n✅ 한국어 TM 정제 완료!")
if ko_result.get("status") != "no_data":
    print(f"   입력: {ko_result.get('input_count', 0):,}개")
    print(f"   출력: {ko_result.get('output_count', 0):,}개")
    print(f"   효율: {ko_result.get('cleaning_efficiency', 0):.1%}")




In [ ]:
# ===================================================================
# Cell 4: 모든 언어 쌍 정제
# ===================================================================

print("\n" + "=" * 60)
print("🌐 모든 언어 쌍 번역 메모리 정제")
print("=" * 60)

# 모든 언어 쌍 일괄 정제
all_results = clean_all_language_pairs()

# 전체 결과 요약
print("\n📊 전체 정제 결과 요약:")
print("-" * 50)
for lang_suffix, result in all_results.items():
    if result.get("status") == "error":
        print(f"❌ {lang_suffix}: {result.get('error', 'Unknown error')}")
    elif result.get("status") == "no_data":
        print(f"⚠️ {lang_suffix}: Raw TM 데이터 없음")
    else:
        input_count = result.get('input_count', 0)
        output_count = result.get('output_count', 0)
        efficiency = result.get('cleaning_efficiency', 0)
        print(f"✅ {lang_suffix}: {input_count:,} → {output_count:,} ({efficiency:.1%})")




In [ ]:
# ===================================================================
# Cell 5: 정제 결과 비교 및 품질 확인
# ===================================================================

def compare_before_after(lang_suffix: str, limit: int = 3):
    """정제 전후 비교"""
    client = MongoClient(MONGO_CONNECTION_STRING)
    db = client[DB_NAME]
    clean_tm_collection = db[f"clean_translation_memory_{lang_suffix}"]
    
    count = clean_tm_collection.count_documents({})
    if count == 0:
        print(f"⚠️ [{lang_suffix}] 정제된 TM이 없습니다.")
        return
    
    print(f"🔍 [{lang_suffix}] 정제 전후 비교 (총 {count:,}개 중 {limit}개 샘플):")
    samples = list(clean_tm_collection.find({}).limit(limit))
    
    for i, sample in enumerate(samples, 1):
        print(f"\n--- 샘플 {i} ---")
        print(f"✨ 정제 후:")
        print(f"   Source: '{sample.get('source_text', '')}'")
        print(f"   Target: '{sample.get('target_text', '')}'")
        print(f"🗑️ 원본:")
        print(f"   Source: '{sample.get('original_source_text', '')[:100]}...'")
        print(f"   Target: '{sample.get('original_target_text', '')[:100]}...'")
        print(f"📊 품질: {sample.get('quality_score', 'N/A'):.2f} ({sample.get('text_type', 'N/A')})")

print("📋 정제 결과 품질 확인:")
for source_lang, target_lang in SUPPORTED_LANGUAGE_PAIRS:
    lang_suffix = f"{source_lang}_{target_lang}"
    print(f"\n--- {source_lang.upper()}-{target_lang.upper()} ---")
    compare_before_after(lang_suffix, limit=2)




In [ ]:
# ===================================================================
# Cell 6: 정제 통계 상세 분석
# ===================================================================

print("\n" + "=" * 60)
print("📈 정제 통계 상세 분석")
print("=" * 60)

# 각 언어 쌍의 상세 통계
for source_lang, target_lang in SUPPORTED_LANGUAGE_PAIRS:
    lang_suffix = f"{source_lang}_{target_lang}"
    stats = get_cleaning_stats(lang_suffix)
    
    if not stats:
        print(f"⚠️ {source_lang.upper()}-{target_lang.upper()}: 통계 데이터 없음")
        continue
    
    print(f"\n📊 {source_lang.upper()}-{target_lang.upper()} 상세 통계:")
    print(f"   🕐 정제 시간: {stats['cleaning_date']}")
    print(f"   📈 효율성: {stats['cleaning_efficiency']:.1%}")
    print(f"   📥 입력: {stats['input_count']:,}개")
    print(f"   📤 출력: {stats['output_count']:,}개")
    print(f"   🗑️ 제거: {stats['removed_count']:,}개")
    
    # 제거 이유 Top 5
    print(f"   🔍 주요 제거 이유:")
    removal_reasons = sorted(stats['removal_reasons'].items(), key=lambda x: x[1], reverse=True)
    for reason, count in removal_reasons[:5]:
        percentage = count / stats['input_count'] * 100
        print(f"     - {reason}: {count:,}개 ({percentage:.1f}%)")
    
    # 품질 분포
    print(f"   🏆 품질 분포:")
    quality_dist = stats['quality_distribution']
    for tier in ['high', 'medium', 'low', 'very_low']:
        count = quality_dist.get(tier, 0)
        if count > 0:
            percentage = count / stats['output_count'] * 100
            print(f"     - {tier}: {count:,}개 ({percentage:.1f}%)")
    
    # 텍스트 유형 Top 3
    print(f"   📝 주요 텍스트 유형:")
    text_types = sorted(stats['text_types'].items(), key=lambda x: x[1], reverse=True)
    for text_type, count in text_types[:3]:
        percentage = count / stats['output_count'] * 100
        print(f"     - {text_type}: {count:,}개 ({percentage:.1f}%)")




In [ ]:
# ===================================================================
# Cell 7: MongoDB 컬렉션 현황 확인
# ===================================================================

print("\n" + "=" * 60)
print("🗄️ MongoDB 컬렉션 현황")
print("=" * 60)

# MongoDB 연결 및 컬렉션 목록 조회
client = MongoClient(MONGO_CONNECTION_STRING)
db = client[DB_NAME]
collections = db.list_collection_names()

# TM 관련 컬렉션 분류
raw_tm_collections = [c for c in collections if c.startswith("translation_memory_") and not c.startswith("clean_")]
clean_tm_collections = [c for c in collections if c.startswith("clean_translation_memory_")]
stats_collections = [c for c in collections if c.startswith("tm_cleaning_stats_")]

print(f"📋 TM 관련 컬렉션 현황:")
print(f"   📄 Raw TM: {len(raw_tm_collections)}개")
print(f"   ✨ Clean TM: {len(clean_tm_collections)}개")
print(f"   📊 통계: {len(stats_collections)}개")

# 컬렉션별 문서 수와 저장 공간 정보
print(f"\n📊 컬렉션별 상세 정보:")
print(f"{'타입':<12} {'컬렉션명':<35} {'문서수':<12} {'상태'}")
print("-" * 70)

for collection_name in sorted(raw_tm_collections + clean_tm_collections + stats_collections):
    count = db[collection_name].count_documents({})
    
    if collection_name.startswith("translation_memory_"):
        col_type = "Raw TM"
    elif collection_name.startswith("clean_translation_memory_"):
        col_type = "Clean TM"
    else:
        col_type = "통계"
    
    status = "✅ 정상" if count > 0 else "⚠️ 비어있음"
    print(f"{col_type:<12} {collection_name:<35} {count:>8,}개 {status}")

# 정제 효율성 요약
print(f"\n📈 전체 정제 효율성 요약:")
total_raw = sum(db[c].count_documents({}) for c in raw_tm_collections)
total_clean = sum(db[c].count_documents({}) for c in clean_tm_collections)

if total_raw > 0:
    overall_efficiency = total_clean / total_raw
    print(f"   전체 Raw TM: {total_raw:,}개")
    print(f"   전체 Clean TM: {total_clean:,}개")
    print(f"   전체 효율성: {overall_efficiency:.1%}")
    print(f"   제거된 노이즈: {total_raw - total_clean:,}개")
else:
    print("   ⚠️ Raw TM 데이터가 없습니다.")

print("\n✅ TM 정제 작업 및 분석 완료!")
print("💡 이제 clean_translation_memory_* 컬렉션을 사용하여 AI 검색 인덱스를 업데이트하세요.")




In [ ]:
# ===================================================================
# Cell 8: 정제된 TM 샘플 확인 (최종 검증)
# ===================================================================

print("\n" + "=" * 60)
print("🔍 정제된 TM 최종 검증")
print("=" * 60)

def verify_clean_tm_quality(lang_suffix: str, sample_size: int = 10):
    """정제된 TM 품질 검증"""
    client = MongoClient(MONGO_CONNECTION_STRING)
    db = client[DB_NAME]
    clean_tm_collection = db[f"clean_translation_memory_{lang_suffix}"]
    
    print(f"🔍 [{lang_suffix}] 정제된 TM 품질 검증:")
    
    # 랜덤 샘플 추출
    pipeline = [{"$sample": {"size": sample_size}}]
    samples = list(clean_tm_collection.aggregate(pipeline))
    
    if not samples:
        print("   ⚠️ 정제된 TM이 없습니다.")
        return
    
    # 품질 검증
    high_quality = 0
    has_meaningful_text = 0
    proper_length = 0
    
    print(f"   📋 {len(samples)}개 샘플 검증 결과:")
    
    for sample in samples:
        source = sample.get('source_text', '')
        target = sample.get('target_text', '')
        quality = sample.get('quality_score', 0)
        
        # 고품질 체크
        if quality >= 0.5:
            high_quality += 1
        
        # 의미있는 텍스트 체크
        if len(source.strip()) > 2 and len(target.strip()) > 2:
            has_meaningful_text += 1
        
        # 적절한 길이 체크
        if 2 <= len(source.split()) <= 100:
            proper_length += 1
    
    print(f"     ✅ 고품질 (≥0.5): {high_quality}/{len(samples)} ({high_quality/len(samples)*100:.1f}%)")
    print(f"     ✅ 의미있는 텍스트: {has_meaningful_text}/{len(samples)} ({has_meaningful_text/len(samples)*100:.1f}%)")
    print(f"     ✅ 적절한 길이: {proper_length}/{len(samples)} ({proper_length/len(samples)*100:.1f}%)")
    
    # 우수 샘플 3개 표시
    high_quality_samples = [s for s in samples if s.get('quality_score', 0) >= 0.7]
    if high_quality_samples:
        print(f"   🏆 우수 품질 샘플:")
        for i, sample in enumerate(high_quality_samples[:3], 1):
            print(f"     {i}. '{sample.get('source_text', '')}' → '{sample.get('target_text', '')}'")
            print(f"        (품질: {sample.get('quality_score', 0):.2f}, 유형: {sample.get('text_type', 'N/A')})")

# 각 언어 쌍 검증
for source_lang, target_lang in SUPPORTED_LANGUAGE_PAIRS:
    lang_suffix = f"{source_lang}_{target_lang}"
    verify_clean_tm_quality(lang_suffix, sample_size=20)
    print()

print("🎉 TM 정제 프로세스 완료!")
print("📝 다음 단계: ChromaDB 인덱스를 정제된 TM으로 업데이트")

In [10]:
# ===================================================================
# Cell 2: Clean TM을 CSV로 내보내기 함수
# ===================================================================

def export_clean_tm_to_csv(lang_suffix: str, include_metadata: bool = True) -> str:
    """정제된 TM을 CSV 파일로 내보내기"""
    
    client = MongoClient(MONGO_CONNECTION_STRING)
    db = client[DB_NAME]
    clean_tm_collection = db[f"clean_translation_memory_{lang_suffix}"]
    
    print(f"📤 [{lang_suffix}] Clean TM CSV 내보내기 시작...")
    
    # 데이터 조회
    cursor = clean_tm_collection.find({})
    docs = list(cursor)
    
    if not docs:
        print(f"   ⚠️ {lang_suffix} Clean TM 데이터가 없습니다.")
        return None
    
    print(f"   📊 총 {len(docs):,}개 문서 로드 완료")
    
    # DataFrame 생성용 데이터 준비
    csv_data = []
    
    for doc in docs:
        row = {
            'source_text': doc.get('source_text', ''),
            'target_text': doc.get('target_text', ''),
            'quality_score': doc.get('quality_score', 0),
            'text_type': doc.get('text_type', ''),
            'word_count_source': doc.get('word_count_source', 0),
            'word_count_target': doc.get('word_count_target', 0),
        }
        
        if include_metadata:
            row.update({
                'page_path': doc.get('page_path', ''),
                'component_path': doc.get('component_path', ''),
                'component_type': doc.get('component_type', ''),
                'version_name_source': doc.get('version_name', ''),  # 소스 버전
                'version_number': doc.get('version_number', 1),
                'original_source_text': doc.get('original_source_text', ''),
                'original_target_text': doc.get('original_target_text', ''),
                'cleaning_method': doc.get('cleaning_method', ''),
                'cleaned_at': doc.get('cleaned_at', '')
            })
        
        csv_data.append(row)
    
    # DataFrame 생성
    df = pd.DataFrame(csv_data)
    
    # 품질 순으로 정렬
    df = df.sort_values(['quality_score', 'word_count_source'], ascending=[False, False])
    
    # CSV 파일명 생성
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    metadata_suffix = "_with_metadata" if include_metadata else "_simple"
    filename = f"clean_tm_{lang_suffix}{metadata_suffix}_{timestamp}.csv"
    filepath = os.path.join(csv_output_dir, filename)
    
    # CSV 저장
    df.to_csv(filepath, index=False, encoding='utf-8-sig')
    
    print(f"   ✅ CSV 저장 완료: {filename}")
    print(f"   📊 컬럼 수: {len(df.columns)}개")
    print(f"   📋 행 수: {len(df):,}개")
    print(f"   💾 파일 크기: {os.path.getsize(filepath) / 1024 / 1024:.1f} MB")
    
    return filepath

def export_tm_statistics_to_csv() -> str:
    """TM 정제 통계를 CSV로 내보내기"""
    
    client = MongoClient(MONGO_CONNECTION_STRING)
    db = client[DB_NAME]
    
    print("📊 TM 정제 통계 CSV 내보내기...")
    
    stats_data = []
    
    for source_lang, target_lang in SUPPORTED_LANGUAGE_PAIRS:
        lang_suffix = f"{source_lang}_{target_lang}"
        stats_collection = db[f"tm_cleaning_stats_{lang_suffix}"]
        
        stats_doc = stats_collection.find_one({}, sort=[("cleaning_date", -1)])
        
        if stats_doc:
            # 제거 이유를 개별 컬럼으로 분리
            removal_reasons = stats_doc.get('removal_reasons', {})
            quality_dist = stats_doc.get('quality_distribution', {})
            text_types = stats_doc.get('text_types', {})
            
            row = {
                'language_pair': lang_suffix,
                'source_language': source_lang,
                'target_language': target_lang,
                'cleaning_date': stats_doc.get('cleaning_date', ''),
                'input_count': stats_doc.get('input_count', 0),
                'output_count': stats_doc.get('output_count', 0),
                'removed_count': stats_doc.get('removed_count', 0),
                'cleaning_efficiency': stats_doc.get('cleaning_efficiency', 0),
                
                # 제거 이유별 카운트
                'removed_empty_source_text': removal_reasons.get('empty_source_text', 0),
                'removed_empty_target_text': removal_reasons.get('empty_target_text', 0),
                'removed_source_too_short': removal_reasons.get('source_too_short', 0),
                'removed_target_too_short': removal_reasons.get('target_too_short', 0),
                'removed_jcr_sling_properties': removal_reasons.get('jcr_sling_properties', 0),
                'removed_no_meaningful_chars': removal_reasons.get('source_no_meaningful_chars', 0) + removal_reasons.get('target_no_meaningful_chars', 0),
                'removed_only_numbers_symbols': removal_reasons.get('source_only_numbers_symbols', 0) + removal_reasons.get('target_only_numbers_symbols', 0),
                
                # 품질 분포
                'quality_high': quality_dist.get('high', 0),
                'quality_medium': quality_dist.get('medium', 0),
                'quality_low': quality_dist.get('low', 0),
                'quality_very_low': quality_dist.get('very_low', 0),
                
                # 텍스트 유형 (주요 3개)
                'text_type_title': text_types.get('title', 0),
                'text_type_sentence': text_types.get('sentence', 0),
                'text_type_plain_text': text_types.get('plain_text', 0),
                'text_type_others': sum(text_types.values()) - text_types.get('title', 0) - text_types.get('sentence', 0) - text_types.get('plain_text', 0)
            }
            
            stats_data.append(row)
    
    if not stats_data:
        print("   ⚠️ 정제 통계 데이터가 없습니다.")
        return None
    
    # DataFrame 생성 및 저장
    df = pd.DataFrame(stats_data)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"tm_cleaning_statistics_{timestamp}.csv"
    filepath = os.path.join(csv_output_dir, filename)
    
    df.to_csv(filepath, index=False, encoding='utf-8-sig')
    
    print(f"   ✅ 통계 CSV 저장 완료: {filename}")
    print(f"   📊 언어 쌍 수: {len(df)}개")
    
    return filepath




In [11]:
# ===================================================================
# Cell 3: 품질별 샘플 CSV 생성
# ===================================================================

def export_quality_samples_to_csv(lang_suffix: str, samples_per_tier: int = 50) -> str:
    """품질 등급별 샘플을 CSV로 내보내기"""
    
    client = MongoClient(MONGO_CONNECTION_STRING)
    db = client[DB_NAME]
    clean_tm_collection = db[f"clean_translation_memory_{lang_suffix}"]
    
    print(f"🏆 [{lang_suffix}] 품질별 샘플 CSV 생성...")
    
    quality_tiers = ['high', 'medium', 'low', 'very_low']
    all_samples = []
    
    for tier in quality_tiers:
        print(f"   🔍 {tier} 품질 샘플 추출 중...")
        
        # 해당 품질 등급의 문서들을 무작위로 샘플링
        pipeline = [
            {"$match": {"quality_score": {"$exists": True}}},
            {"$addFields": {
                "quality_tier": {
                    "$switch": {
                        "branches": [
                            {"case": {"$gte": ["$quality_score", 0.8]}, "then": "high"},
                            {"case": {"$gte": ["$quality_score", 0.5]}, "then": "medium"},
                            {"case": {"$gte": ["$quality_score", 0.2]}, "then": "low"}
                        ],
                        "default": "very_low"
                    }
                }
            }},
            {"$match": {"quality_tier": tier}},
            {"$sample": {"size": samples_per_tier}},
            {"$sort": {"quality_score": -1}}
        ]
        
        samples = list(clean_tm_collection.aggregate(pipeline))
        
        for sample in samples:
            sample_data = {
                'quality_tier': tier,
                'quality_score': sample.get('quality_score', 0),
                'source_text': sample.get('source_text', ''),
                'target_text': sample.get('target_text', ''),
                'text_type': sample.get('text_type', ''),
                'word_count_source': sample.get('word_count_source', 0),
                'word_count_target': sample.get('word_count_target', 0),
                'page_path': sample.get('page_path', ''),
                'component_type': sample.get('component_type', ''),
                'original_source_text': sample.get('original_source_text', '')[:200],  # 원본은 200자만
                'original_target_text': sample.get('original_target_text', '')[:200]   # 원본은 200자만
            }
            all_samples.append(sample_data)
        
        print(f"     ✅ {len(samples)}개 샘플 추출")
    
    if not all_samples:
        print(f"   ⚠️ {lang_suffix} 품질 샘플이 없습니다.")
        return None
    
    # DataFrame 생성 및 저장
    df = pd.DataFrame(all_samples)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"clean_tm_quality_samples_{lang_suffix}_{timestamp}.csv"
    filepath = os.path.join(csv_output_dir, filename)
    
    df.to_csv(filepath, index=False, encoding='utf-8-sig')
    
    print(f"   ✅ 품질 샘플 CSV 저장 완료: {filename}")
    print(f"   📊 총 샘플 수: {len(df):,}개")
    
    # 품질별 분포 출력
    tier_counts = df['quality_tier'].value_counts()
    for tier in quality_tiers:
        count = tier_counts.get(tier, 0)
        print(f"     - {tier}: {count}개")
    
    return filepath




In [12]:
# ===================================================================
# Cell 4: 전체 Clean TM CSV 내보내기 실행
# ===================================================================

print("🚀 정제된 TM CSV 내보내기 시작")
print("=" * 50)

exported_files = []

# 1. 각 언어 쌍의 Clean TM 내보내기
for source_lang, target_lang in SUPPORTED_LANGUAGE_PAIRS:
    lang_suffix = f"{source_lang}_{target_lang}"
    
    print(f"\n📤 {source_lang.upper()}-{target_lang.upper()} TM 내보내기...")
    
    # 메타데이터 포함 버전
    filepath_full = export_clean_tm_to_csv(lang_suffix, include_metadata=True)
    if filepath_full:
        exported_files.append(filepath_full)
    
    # 간단 버전 (번역쌍 + 품질 정보만)
    filepath_simple = export_clean_tm_to_csv(lang_suffix, include_metadata=False)
    if filepath_simple:
        exported_files.append(filepath_simple)

# 2. 정제 통계 내보내기
print(f"\n📊 정제 통계 내보내기...")
stats_filepath = export_tm_statistics_to_csv()
if stats_filepath:
    exported_files.append(stats_filepath)

print(f"\n✅ CSV 내보내기 완료!")
print(f"📁 출력 경로: {csv_output_dir}")
print(f"📄 생성된 파일: {len(exported_files)}개")

for filepath in exported_files:
    filename = os.path.basename(filepath)
    size_mb = os.path.getsize(filepath) / 1024 / 1024
    print(f"   - {filename} ({size_mb:.1f} MB)")




🚀 정제된 TM CSV 내보내기 시작

📤 EN-KO TM 내보내기...
📤 [en_ko] Clean TM CSV 내보내기 시작...
   📊 총 13,743개 문서 로드 완료
   ✅ CSV 저장 완료: clean_tm_en_ko_with_metadata_20250812_111944.csv
   📊 컬럼 수: 15개
   📋 행 수: 13,743개
   💾 파일 크기: 12.0 MB
📤 [en_ko] Clean TM CSV 내보내기 시작...
   📊 총 13,743개 문서 로드 완료
   ✅ CSV 저장 완료: clean_tm_en_ko_simple_20250812_111944.csv
   📊 컬럼 수: 6개
   📋 행 수: 13,743개
   💾 파일 크기: 4.3 MB

📤 EN-JA TM 내보내기...
📤 [en_ja] Clean TM CSV 내보내기 시작...
   📊 총 14,492개 문서 로드 완료
   ✅ CSV 저장 완료: clean_tm_en_ja_with_metadata_20250812_111945.csv
   📊 컬럼 수: 15개
   📋 행 수: 14,492개
   💾 파일 크기: 14.0 MB
📤 [en_ja] Clean TM CSV 내보내기 시작...
   📊 총 14,492개 문서 로드 완료
   ✅ CSV 저장 완료: clean_tm_en_ja_simple_20250812_111945.csv
   📊 컬럼 수: 6개
   📋 행 수: 14,492개
   💾 파일 크기: 5.1 MB

📊 정제 통계 내보내기...
📊 TM 정제 통계 CSV 내보내기...
   ✅ 통계 CSV 저장 완료: tm_cleaning_statistics_20250812_111945.csv
   📊 언어 쌍 수: 2개

✅ CSV 내보내기 완료!
📁 출력 경로: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/3_processed/clean_tm_csv
📄 생성된 파일: 5개

In [13]:
# ===================================================================
# Cell 5: 품질별 샘플 CSV 생성
# ===================================================================

print("\n" + "=" * 50)
print("🏆 품질별 샘플 CSV 생성")
print("=" * 50)

sample_files = []

for source_lang, target_lang in SUPPORTED_LANGUAGE_PAIRS:
    lang_suffix = f"{source_lang}_{target_lang}"
    
    print(f"\n🔍 {source_lang.upper()}-{target_lang.upper()} 품질 샘플...")
    
    sample_filepath = export_quality_samples_to_csv(lang_suffix, samples_per_tier=30)
    if sample_filepath:
        sample_files.append(sample_filepath)

print(f"\n✅ 품질 샘플 CSV 생성 완료!")
for filepath in sample_files:
    filename = os.path.basename(filepath)
    size_mb = os.path.getsize(filepath) / 1024 / 1024
    print(f"   - {filename} ({size_mb:.1f} MB)")





🏆 품질별 샘플 CSV 생성

🔍 EN-KO 품질 샘플...
🏆 [en_ko] 품질별 샘플 CSV 생성...
   🔍 high 품질 샘플 추출 중...
     ✅ 30개 샘플 추출
   🔍 medium 품질 샘플 추출 중...
     ✅ 30개 샘플 추출
   🔍 low 품질 샘플 추출 중...
     ✅ 30개 샘플 추출
   🔍 very_low 품질 샘플 추출 중...
     ✅ 0개 샘플 추출
   ✅ 품질 샘플 CSV 저장 완료: clean_tm_quality_samples_en_ko_20250812_112206.csv
   📊 총 샘플 수: 90개
     - high: 30개
     - medium: 30개
     - low: 30개
     - very_low: 0개

🔍 EN-JA 품질 샘플...
🏆 [en_ja] 품질별 샘플 CSV 생성...
   🔍 high 품질 샘플 추출 중...
     ✅ 30개 샘플 추출
   🔍 medium 품질 샘플 추출 중...
     ✅ 30개 샘플 추출
   🔍 low 품질 샘플 추출 중...
     ✅ 30개 샘플 추출
   🔍 very_low 품질 샘플 추출 중...
     ✅ 0개 샘플 추출
   ✅ 품질 샘플 CSV 저장 완료: clean_tm_quality_samples_en_ja_20250812_112206.csv
   📊 총 샘플 수: 90개
     - high: 30개
     - medium: 30개
     - low: 30개
     - very_low: 0개

✅ 품질 샘플 CSV 생성 완료!
   - clean_tm_quality_samples_en_ko_20250812_112206.csv (0.0 MB)
   - clean_tm_quality_samples_en_ja_20250812_112206.csv (0.2 MB)


In [14]:
# ===================================================================
# Cell 6: CSV 파일 내용 미리보기
# ===================================================================

print("\n" + "=" * 50)
print("👀 생성된 CSV 파일 미리보기")
print("=" * 50)

def preview_csv_file(filepath: str, rows: int = 5):
    """CSV 파일 미리보기"""
    filename = os.path.basename(filepath)
    print(f"\n📄 {filename}")
    
    try:
        df = pd.read_csv(filepath)
        print(f"   📊 크기: {len(df):,}행 × {len(df.columns)}열")
        print(f"   📋 컬럼: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
        
        print(f"   🔍 처음 {rows}행:")
        for i, (_, row) in enumerate(df.head(rows).iterrows()):
            source = str(row.get('source_text', ''))[:50]
            target = str(row.get('target_text', ''))[:50]
            quality = row.get('quality_score', 'N/A')
            print(f"     {i+1}. [{quality}] '{source}...' → '{target}...'")
            
    except Exception as e:
        print(f"   ❌ 미리보기 오류: {str(e)}")

# 주요 파일들 미리보기
preview_files = [f for f in exported_files if 'with_metadata' in f]  # 메타데이터 포함 파일만
preview_files.extend(sample_files)  # 샘플 파일들

for filepath in preview_files[:4]:  # 최대 4개 파일만 미리보기
    preview_csv_file(filepath)

print(f"\n🎉 모든 CSV 내보내기 작업 완료!")
print(f"📂 파일 위치: {csv_output_dir}")
print(f"💡 Excel이나 텍스트 에디터로 파일을 열어서 정제 품질을 확인해보세요.")


👀 생성된 CSV 파일 미리보기

📄 clean_tm_en_ko_with_metadata_20250812_111944.csv
   📊 크기: 13,743행 × 15열
   📋 컬럼: source_text, target_text, quality_score, text_type, word_count_source...
   🔍 처음 5행:
     1. [1.0] 'Circulating tumor DNA (ctDNA) may represent a very...' → '순환 종양 DNA(circulating tumor DNA, ctDNA)는 차세대 염기서열분...'
     2. [1.0] 'Putting people first and doing the right thing are...' → '옳은 일을 하는 것은 Illumina의 정체성과 Illumina 사업의 핵심입니다. Ill...'
     3. [1.0] 'Compute pricing rates for Illumina Connected Analy...' → 'Illumina Connected Analytics의 컴퓨팅 가격 책정 비율은 ICA 도움...'
     4. [1.0] 'If you currently own a HiSeq 2500, HiSeq 3000/4000...' → '현재 HiSeq 2500, HiSeq 3000/4000 또는 HiSeq X를 소유하고 있다...'
     5. [1.0] 'Run the RNA-Seq workflow (FASTQ only) on the MiSeq...' → 'MiSeq System에서 RNA-Seq 워크플로우(FASTQ만 해당)를 실행하고 Base...'

📄 clean_tm_en_ja_with_metadata_20250812_111945.csv
   📊 크기: 14,492행 × 15열
   📋 컬럼: source_text, target_text, quality_score, text_type, word_count_source...
   🔍 처음 5행:
   